In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_absolute_percentage_error
from sklearn.metrics import mean_squared_error

In [ ]:
from keras.models import Sequential
from keras.layers import GRU
from keras.layers import Dropout
from keras.layers import Dense

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))

In [ ]:

def split(size, obs):
    x = []
    y = []

    for i in range(len(obs)-SEQUENCE_SIZE):
        #print(i)
        window = obs[i:(i+SEQUENCE_SIZE)]
        after_window = obs[i+SEQUENCE_SIZE]
        window = [[x] for x in window]
        #print("{} - {}".format(window,after_window))
        x.append(window)
        y.append(after_window)

    return np.array(x),np.array(y)





# **...... BITCOIN......**

In [ ]:
btcpath='/content/BTC-USD (6).csv'


In [ ]:
btc=pd.read_csv(btcpath)

In [ ]:
btc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-01-01,963.658020,1003.080017,958.698975,998.325012,998.325012,147775008
1,2017-01-02,998.617004,1031.390015,996.702026,1021.750000,1021.750000,222184992
2,2017-01-03,1021.599976,1044.079956,1021.599976,1043.839966,1043.839966,185168000
3,2017-01-04,1044.400024,1159.420044,1044.400024,1154.729980,1154.729980,344945984
4,2017-01-05,1156.729980,1191.099976,910.416992,1013.380005,1013.380005,510199008
...,...,...,...,...,...,...,...
2474,2023-10-11,27392.076172,27474.115234,26561.099609,26873.320313,26873.320313,13648094333
2475,2023-10-12,26873.292969,26921.439453,26558.320313,26756.798828,26756.798828,9392909295
2476,2023-10-13,26752.878906,27092.697266,26686.322266,26862.375000,26862.375000,15165312851
2477,2023-10-14,26866.203125,26969.000000,26814.585938,26861.707031,26861.707031,5388116782


In [ ]:
btc_close=btc.iloc[: , 4:5].values

In [ ]:
btc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2479 entries, 0 to 2478
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2479 non-null   object 
 1   Open       2479 non-null   float64
 2   High       2479 non-null   float64
 3   Low        2479 non-null   float64
 4   Close      2479 non-null   float64
 5   Adj Close  2479 non-null   float64
 6   Volume     2479 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 135.7+ KB


In [ ]:
btc_close

array([[  998.325012],
       [ 1021.75    ],
       [ 1043.839966],
       ...,
       [26862.375   ],
       [26861.707031],
       [27024.675781]])

In [ ]:
btc_close=scaler.fit_transform(btc_close)

In [ ]:
btc_close

array([[0.00330246],
       [0.00365319],
       [0.00398393],
       ...,
       [0.39055219],
       [0.39054219],
       [0.39298224]])

In [ ]:
# Split Data........

btc_train_size = int(len(btc_close)*.8 )
btc_train=btc_close[:btc_train_size]
btc_test=btc_close[btc_train_size:]

In [ ]:
SEQUENCE_SIZE=30
btcx_train,btcy_train = split(SEQUENCE_SIZE,btc_train)
btcx_test,btcy_test = split(SEQUENCE_SIZE,btc_test)

print("Shape of training set: {}".format(btcx_train.shape))
print("Shape of test set: {}".format(btcx_test.shape))

Shape of training set: (1953, 30, 1, 1)
Shape of test set: (466, 30, 1, 1)


In [ ]:
btcx_train = btcx_train.reshape( btcx_train.shape[0], btcx_train.shape[1], 1)

In [ ]:
btcx_train.shape

(1953, 30, 1)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(btcx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=120,return_sequences=False))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru (GRU)                   (None, 30, 30)            2970      
                                                                 
 dropout (Dropout)           (None, 30, 30)            0         
                                                                 
 gru_1 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_1 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_2 (GRU)                 (None, 30, 90)            41040     
                                                                 
 dropout_2 (Dropout)         (None, 30, 90)            0         
                                                                 
 gru_3 (GRU)                 (None, 120)               7

In [ ]:
model.fit(btcx_train,btcy_train,epochs=100,batch_size=32)

Epoch 1/100
62/62 [==============================] - 14s 82ms/step - loss: 0.0103
Epoch 2/100
62/62 [==============================] - 7s 109ms/step - loss: 0.0026
Epoch 3/100
62/62 [==============================] - 5s 81ms/step - loss: 0.0026
Epoch 4/100
62/62 [==============================] - 5s 86ms/step - loss: 0.0019
Epoch 5/100
62/62 [==============================] - 6s 96ms/step - loss: 0.0016
Epoch 6/100
62/62 [==============================] - 5s 80ms/step - loss: 0.0016
Epoch 7/100
62/62 [==============================] - 7s 109ms/step - loss: 0.0023
Epoch 8/100
62/62 [==============================] - 5s 78ms/step - loss: 0.0016
Epoch 9/100
62/62 [==============================] - 5s 80ms/step - loss: 0.0015
Epoch 10/100
62/62 [==============================] - 7s 108ms/step - loss: 0.0015
Epoch 11/100
62/62 [==============================] - 5s 79ms/step - loss: 0.0015
Epoch 12/100
62/62 [==============================] - 8s 126ms/step - loss: 0.0013
Epoch 13/100
62/62 [

In [ ]:
btcy_pred=model.predict(btcx_test)

15/15 [==============================] - 3s 35ms/step


In [ ]:

print('MAE =',mean_absolute_error(btcy_test,btcy_pred))


MAE = 0.014145147771303643


In [ ]:
print('MAPE =',mean_absolute_percentage_error(btcy_test,btcy_pred))

MAPE = 0.04169708575259211


In [ ]:
print('MSE=',mean_squared_error(btcy_test,btcy_pred))

MSE= 0.00026934359551592456


In [ ]:
#Root Mean Sqquare Erorr
print('RMSE=',np.sqrt(mean_squared_error(btcy_test,btcy_pred)))

RMSE= 0.01641169081831377


# **......LETCOIN......**

In [ ]:
ltcpath='/content/LTC-USD (1).csv'


In [ ]:
ltc=pd.read_csv(ltcpath)

In [ ]:
ltc

,Date,Open,High,Low,Close,Adj Close,Volume
0,2014-09-18,5.065430,5.065430,4.579960,4.685230,4.685230,4569260
1,2014-09-19,4.687290,4.755820,4.254350,4.327770,4.327770,3917450
2,2014-09-20,4.329200,4.616080,4.202190,4.286440,4.286440,5490660
3,2014-09-21,4.263070,4.300130,4.154990,4.245920,4.245920,2931220
4,2014-09-22,4.245930,4.416880,4.210130,4.242350,4.242350,1855960
...,...,...,...,...,...,...,...
3310,2023-10-11,63.619778,63.685921,61.262772,61.499821,61.499821,186389594
3311,2023-10-12,61.501141,61.521873,60.754681,61.127815,61.127815,171888213
3312,2023-10-13,61.127197,61.995136,61.112534,61.574059,61.574059,203849754
3313,2023-10-14,61.574585,61.949982,61.470036,61.665157,61.665157,120785909


In [ ]:
ltc_close=ltc.iloc[: , 4:5].values

In [ ]:
ltc.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3315 entries, 0 to 3314
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       3315 non-null   object 
 1   Open       3315 non-null   float64
 2   High       3315 non-null   float64
 3   Low        3315 non-null   float64
 4   Close      3315 non-null   float64
 5   Adj Close  3315 non-null   float64
 6   Volume     3315 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 181.4+ KB


In [ ]:
ltc_close

array([[ 4.68523 ],
       [ 4.32777 ],
       [ 4.28644 ],
       ...,
       [61.574059],
       [61.665157],
       [61.762432]])

In [ ]:
ltc_close=scaler.fit_transform(ltc_close)

In [ ]:
ltc_close

array([[0.00915722],
       [0.00822946],
       [0.00812219],
       ...,
       [0.15680775],
       [0.15704419],
       [0.15729666]])

In [ ]:
ltc_train_size = int(len(ltc_close)*.8 )
ltc_train=ltc_close[:ltc_train_size]
ltc_test=ltc_close[ltc_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ltcx_train,ltcy_train = split(SEQUENCE_SIZE,ltc_train)
ltcx_test,ltcy_test = split(SEQUENCE_SIZE,ltc_test)

print("Shape of training set: {}".format(ltcx_train.shape))
print("Shape of test set: {}".format(ltcx_test.shape))

Shape of training set: (2622, 30, 1, 1)
Shape of test set: (633, 30, 1, 1)


In [ ]:
ltcx_train = ltcx_train.reshape( ltcx_train.shape[0], ltcx_train.shape[1], 1)

In [ ]:
ltcx_train.shape

(2622, 30, 1)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(ltcx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_4 (GRU)                 (None, 30, 30)            2970      
                                                                 
 dropout_4 (Dropout)         (None, 30, 30)            0         
                                                                 
 gru_5 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_5 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_6 (GRU)                 (None, 90)                41040     
                                                                 
 dropout_6 (Dropout)         (None, 90)                0         
                                                                 
 dense_1 (Dense)             (None, 1)                

In [ ]:
model.fit(ltcx_train,ltcy_train,epochs=100,batch_size=30)

Epoch 1/100
88/88 [==============================] - 12s 72ms/step - loss: 0.0051
Epoch 2/100
88/88 [==============================] - 5s 52ms/step - loss: 0.0013
Epoch 3/100
88/88 [==============================] - 4s 49ms/step - loss: 0.0013
Epoch 4/100
88/88 [==============================] - 6s 63ms/step - loss: 0.0015
Epoch 5/100
88/88 [==============================] - 4s 50ms/step - loss: 0.0011
Epoch 6/100
88/88 [==============================] - 4s 47ms/step - loss: 0.0010
Epoch 7/100
88/88 [==============================] - 6s 66ms/step - loss: 0.0012
Epoch 8/100
88/88 [==============================] - 4s 50ms/step - loss: 0.0012
Epoch 9/100
88/88 [==============================] - 5s 55ms/step - loss: 9.5464e-04
Epoch 10/100
88/88 [==============================] - 6s 64ms/step - loss: 9.8750e-04
Epoch 11/100
88/88 [==============================] - 5s 52ms/step - loss: 8.4230e-04
Epoch 12/100
88/88 [==============================] - 6s 64ms/step - loss: 8.5593e-04
Epoch 13

In [ ]:
ltcy_pred=model.predict(ltcx_test)

20/20 [==============================] - 2s 16ms/step


In [ ]:

print('MAE =',mean_absolute_error(ltcy_test,ltcy_pred))


MAE = 0.0061135464220387825


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ltcy_test,ltcy_pred))

MAPE = 0.02991507314019749


In [ ]:
print('MSE=',mean_squared_error(ltcy_test,ltcy_pred))

MSE= 7.898093480662659e-05


In [ ]:
#Root Mean Squre Erorr
print('RMSE=',np.sqrt(mean_squared_error(ltcy_test,ltcy_pred)))

RMSE= 0.008887121851681038


# **......ETHEREUM......**

In [ ]:
ethpath='/content/ETH-USD.csv'


In [ ]:
eth=pd.read_csv(ethpath)

In [ ]:
eth

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-12-01,445.209015,472.609009,428.312012,466.540009,466.540009,1247879936
1,2017-12-02,466.851013,476.239014,456.653015,463.449005,463.449005,943649984
2,2017-12-03,463.704987,482.813995,451.851990,465.852997,465.852997,990556992
3,2017-12-04,466.053986,474.777008,453.312012,470.204010,470.204010,1005550016
4,2017-12-05,470.294006,473.558014,457.660004,463.281006,463.281006,1216720000
...,...,...,...,...,...,...,...
2140,2023-10-11,1567.680786,1578.223511,1548.980469,1566.254761,1566.254761,5416504273
2141,2023-10-12,1566.355957,1566.878174,1523.237549,1539.612427,1539.612427,5003930677
2142,2023-10-13,1539.432861,1571.750610,1537.921143,1552.089478,1552.089478,4575141511
2143,2023-10-14,1552.263794,1560.325073,1545.738770,1555.256836,1555.256836,2429214718


In [ ]:
eth_close=eth.iloc[: , 4:5].values

In [ ]:
eth.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2145 entries, 0 to 2144
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   Date       2145 non-null   object 
 1   Open       2145 non-null   float64
 2   High       2145 non-null   float64
 3   Low        2145 non-null   float64
 4   Close      2145 non-null   float64
 5   Adj Close  2145 non-null   float64
 6   Volume     2145 non-null   int64  
dtypes: float64(5), int64(1), object(1)
memory usage: 117.4+ KB


In [ ]:
eth_close

array([[ 466.540009],
       [ 463.449005],
       [ 465.852997],
       ...,
       [1552.089478],
       [1555.256836],
       [1560.048462]])

In [ ]:

eth_close=scaler.fit_transform(eth_close)

In [ ]:
eth_close

array([[0.08084805],
       [0.08019425],
       [0.08070273],
       ...,
       [0.31045892],
       [0.31112886],
       [0.31214237]])

In [ ]:
eth_train_size = int(len(eth_close)*.8 )
eth_train=eth_close[:eth_train_size]
eth_test=eth_close[eth_train_size:]

In [ ]:
SEQUENCE_SIZE=30
ethx_train,ethy_train = split(SEQUENCE_SIZE,eth_train)
ethx_test,ethy_test = split(SEQUENCE_SIZE,eth_test)

print("Shape of training set: {}".format(ethx_train.shape))
print("Shape of test set: {}".format(ethx_test.shape))

Shape of training set: (1686, 30, 1, 1)
Shape of test set: (399, 30, 1, 1)


In [ ]:
ethx_train = ethx_train.reshape( ethx_train.shape[0], ethx_train.shape[1], 1)

In [ ]:
ethx_train.shape

(1686, 30, 1)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(ethx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_7 (GRU)                 (None, 30, 30)            2970      
                                                                 
 dropout_7 (Dropout)         (None, 30, 30)            0         
                                                                 
 gru_8 (GRU)                 (None, 30, 60)            16560     
                                                                 
 dropout_8 (Dropout)         (None, 30, 60)            0         
                                                                 
 gru_9 (GRU)                 (None, 90)                41040     
                                                                 
 dropout_9 (Dropout)         (None, 90)                0         
                                                                 
 dense_2 (Dense)             (None, 1)                

In [ ]:
model.fit(ethx_train,ethy_train,epochs=100,batch_size=30)

Epoch 1/100
57/57 [==============================] - 10s 50ms/step - loss: 0.0075
Epoch 2/100
57/57 [==============================] - 3s 57ms/step - loss: 0.0021
Epoch 3/100
57/57 [==============================] - 4s 62ms/step - loss: 0.0018
Epoch 4/100
57/57 [==============================] - 4s 69ms/step - loss: 0.0019
Epoch 5/100
57/57 [==============================] - 3s 51ms/step - loss: 0.0018
Epoch 6/100
57/57 [==============================] - 3s 55ms/step - loss: 0.0022
Epoch 7/100
57/57 [==============================] - 3s 61ms/step - loss: 0.0018
Epoch 8/100
57/57 [==============================] - 4s 68ms/step - loss: 0.0017
Epoch 9/100
57/57 [==============================] - 3s 54ms/step - loss: 0.0019
Epoch 10/100
57/57 [==============================] - 3s 56ms/step - loss: 0.0019
Epoch 11/100
57/57 [==============================] - 3s 59ms/step - loss: 0.0015
Epoch 12/100
57/57 [==============================] - 4s 69ms/step - loss: 0.0015
Epoch 13/100
57/57 [====

In [ ]:
ethy_pred=model.predict(ethx_test)

13/13 [==============================] - 1s 14ms/step


In [ ]:
print('MAE =',mean_absolute_error(ethy_test,ethy_pred))


MAE = 0.013341170615966583


In [ ]:
print('MAPE =',mean_absolute_percentage_error(ethy_test,ethy_pred))

MAPE = 0.04199708156790236


In [ ]:
print('MSE=',mean_squared_error(ethy_test,ethy_pred))

MSE= 0.00024718760733033154


In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(ethy_test,ethy_pred)))

RMSE= 0.01572220109686718


# **......XMR.....**

In [ ]:
xmr_path='/content/XMR-USD.csv'

In [ ]:
xmr=pd.read_csv(xmr_path)

In [ ]:
xmr

,Date,Open,High,Low,Close,Adj Close,Volume
0,2017-11-10,121.344002,121.665001,101.757004,105.585999,105.585999,84614000
1,2017-11-11,105.750000,127.106003,103.877998,119.615997,119.615997,107708000
2,2017-11-12,119.597000,133.675003,110.617996,123.856003,123.856003,144948000
3,2017-11-13,128.960007,136.528000,120.921997,123.402000,123.402000,116200000
4,2017-11-14,123.615997,124.200996,118.625999,122.352997,122.352997,53544800
...,...,...,...,...,...,...,...
2161,2023-10-11,152.220093,153.929443,150.970428,153.842102,153.842102,61162383
2162,2023-10-12,153.852783,154.344238,151.021347,152.341812,152.341812,65445853
2163,2023-10-13,152.339523,154.368774,151.051971,153.590515,153.590515,51282283
2164,2023-10-14,153.583801,154.282990,151.850967,152.833160,152.833160,33344438


In [ ]:
xmr_close=xmr.iloc[: , 4:5].values

In [ ]:
xmr_close

array([[105.585999],
       [119.615997],
       [123.856003],
       ...,
       [153.590515],
       [152.83316 ],
       [152.830368]])

In [ ]:
xmr_close=scaler.fit_transform(xmr_close)

In [ ]:
xmr_close

array([[0.16107407],
       [0.19221218],
       [0.20162242],
       ...,
       [0.26761504],
       [0.26593417],
       [0.26592798]])

In [ ]:
xmr_train_size = int(len(xmr_close)*.8 )
xmr_train=xmr_close[:xmr_train_size]
xmr_test=xmr_close[xmr_train_size:]

In [ ]:
SEQUENCE_SIZE=30
xmrx_train,xmry_train = split(SEQUENCE_SIZE,xmr_train)
xmrx_test,xmry_test = split(SEQUENCE_SIZE,xmr_test)

print("Shape of training set: {}".format(xmrx_train.shape))
print("Shape of test set: {}".format(xmrx_test.shape))

Shape of training set: (1702, 30, 1, 1)
Shape of test set: (404, 30, 1, 1)


In [ ]:
xmrx_train = xmrx_train.reshape( xmrx_train.shape[0], xmrx_train.shape[1], 1)

In [ ]:
xmrx_train.shape

(1702, 30, 1)

In [ ]:
model=Sequential()
model.add(GRU(units=30,return_sequences=True,input_shape=(xmrx_train.shape[1],1)))
model.add(Dropout(0.2))
model.add(GRU(units=60,return_sequences=True))
model.add(Dropout(0.2))
model.add(GRU(units=90))
model.add(Dropout(0.2))
model.add(Dense(units=1))
model.compile(optimizer='adam',loss='mean_squared_error')
model.summary()

Model: "sequential_3"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 gru_10 (GRU)                (None, 30, 30)            2970      
                                                                 
 dropout_10 (Dropout)        (None, 30, 30)            0         
                                                                 
 gru_11 (GRU)                (None, 30, 60)            16560     
                                                                 
 dropout_11 (Dropout)        (None, 30, 60)            0         
                                                                 
 gru_12 (GRU)                (None, 90)                41040     
                                                                 
 dropout_12 (Dropout)        (None, 90)                0         
                                                                 
 dense_3 (Dense)             (None, 1)                

In [ ]:
model.fit(xmrx_train,xmry_train,epochs=100,batch_size=30)

Epoch 1/100
57/57 [==============================] - 11s 75ms/step - loss: 0.0077
Epoch 2/100
57/57 [==============================] - 3s 48ms/step - loss: 0.0027
Epoch 3/100
57/57 [==============================] - 3s 48ms/step - loss: 0.0024
Epoch 4/100
57/57 [==============================] - 3s 51ms/step - loss: 0.0022
Epoch 5/100
57/57 [==============================] - 4s 72ms/step - loss: 0.0019
Epoch 6/100
57/57 [==============================] - 3s 61ms/step - loss: 0.0021
Epoch 7/100
57/57 [==============================] - 3s 54ms/step - loss: 0.0020
Epoch 8/100
57/57 [==============================] - 3s 51ms/step - loss: 0.0018
Epoch 9/100
57/57 [==============================] - 4s 72ms/step - loss: 0.0017
Epoch 10/100
57/57 [==============================] - 3s 59ms/step - loss: 0.0016
Epoch 11/100
57/57 [==============================] - 3s 49ms/step - loss: 0.0017
Epoch 12/100
57/57 [==============================] - 3s 54ms/step - loss: 0.0016
Epoch 13/100
57/57 [====

In [ ]:
xmry_pred=model.predict(xmrx_test)

13/13 [==============================] - 1s 15ms/step


In [ ]:
from sklearn.metrics import mean_absolute_error
print('MAE =',mean_absolute_error(xmry_test,xmry_pred))


MAE = 0.005962717320820741


In [ ]:
from sklearn.metrics import mean_absolute_percentage_error
print('MAPE =',mean_absolute_percentage_error(xmry_test,xmry_pred))

MAPE = 0.022845933655941744


In [ ]:
from sklearn.metrics import mean_squared_error
print('MSE=',mean_squared_error(xmry_test,xmry_pred))

MSE= 6.929770338819113e-05


In [ ]:
#Root Mean Square Erorre
print('RMSE=',np.sqrt(mean_squared_error(xmry_test,xmry_pred)))

RMSE= 0.008324524213923047
